In [42]:
!pip install tensorflow-datasets

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [43]:
mnist_data = tfds.load("fashion_mnist")

In [5]:
type(mnist_data)

dict

In [6]:
for item in mnist_data:
    print(item)
    print(type(item))

train
<class 'tensorflow_datasets.core.splits.Split'>
test
<class 'tensorflow_datasets.core.splits.Split'>


In [9]:
mnist_train = tfds.load(name="fashion_mnist", split="train")
assert isinstance(mnist_train, tf.data.Dataset)
print(type(mnist_train))

<class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>


In [10]:
for item in mnist_train.take(1):
    print(type(item))
    print(item.keys())

<class 'dict'>
dict_keys(['image', 'label'])


In [44]:
for item in mnist_train.take(1):
    print(type(item))
    print(item.keys())
    print(item['image'])
    print(item['label'])

In [12]:
mnist_test, info = tfds.load(name="fashion_mnist", with_info="true")
print(info)

tfds.core.DatasetInfo(
    name='fashion_mnist',
    full_name='fashion_mnist/3.0.1',
    description="""
    Fashion-MNIST is a dataset of Zalando's article images consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.
    """,
    homepage='https://github.com/zalandoresearch/fashion-mnist',
    data_path='C:\\Users\\Amanpreet Singh\\tensorflow_datasets\\fashion_mnist\\3.0.1',
    download_size=29.45 MiB,
    dataset_size=36.42 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{DBLP:journals/corr/abs-1708-07747,
      author

In [13]:
data = tfds.load('cats_vs_dogs', split='train', as_supervised=True)

Dataset cats_vs_dogs downloaded and prepared to C:\Users\Amanpreet Singh\tensorflow_datasets\cats_vs_dogs\4.0.0. Subsequent calls will reuse this data.


In [14]:
data = tfds.load('cats_vs_dogs', split='train[:10000]', as_supervised=True)

In [15]:
data = tfds.load('cats_vs_dogs', split='train[:20%]', as_supervised=True)

In [16]:
data = tfds.load('cats_vs_dogs', split='train[-1000:]+train[:1000]', as_supervised=True)

In [17]:
train_data = tfds.load('cats_vs_dogs', split='train[:80%]', as_supervised=True)
validation_data = tfds.load('cats_vs_dogs', split='train[80%:90%]', as_supervised=True)
test_data = tfds.load('cats_vs_dogs', split='train[-10%:]', as_supervised=True)

In [18]:
train_length = [i for i,_ in enumerate(train_data)][-1] + 1
print(train_length)

18610


In [45]:
filename="C:\\Users\\Amanpreet Singh\\tensorflow_datasets\\cats_vs_dogs\\4.0.0\\cats_vs_dogs-train.tfrecord-00000-of-00008"
raw_dataset = tf.data.TFRecordDataset(filename)
for raw_record in raw_dataset.take(1):
    print(repr(raw_record))

In [31]:
!pip install tensorflow-addons

### ETL

In [27]:
import tensorflow as tf
import tensorflow_datasets as tfds
#import tensorflow_addons as tfa

In [28]:
# MODEL DEFINITION START #
model = tf.keras.models.Sequential([
                tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
                tf.keras.layers.MaxPooling2D(2, 2),
                tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                tf.keras.layers.MaxPooling2D(2,2),
                tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                tf.keras.layers.MaxPooling2D(2,2),
                tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                tf.keras.layers.MaxPooling2D(2,2),
                tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                tf.keras.layers.MaxPooling2D(2,2),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(512, activation='relu'),
                tf.keras.layers.Dense(1, activation='sigmoid')
    ])
model.compile(optimizer='Adam', loss='binary_crossentropy',
metrics=['accuracy'])
# MODEL DEFINITION END #

In [29]:
# EXTRACT PHASE START #
data = tfds.load('horses_or_humans', split='train', as_supervised=True)
val_data = tfds.load('horses_or_humans', split='test', as_supervised=True)
# EXTRACT PHASE END

Dataset horses_or_humans downloaded and prepared to C:\Users\Amanpreet Singh\tensorflow_datasets\horses_or_humans\3.0.0. Subsequent calls will reuse this data.


In [32]:
# TRANSFORM PHASE START #
def augmentimages(image, label):
    image = tf.cast(image, tf.float32)
    image = (image/255)
    image = tf.image.random_flip_left_right(image)
    image = tfa.image.rotate(image, 40, interpolation='NEAREST')
    return image, label
#train = data.map(augmentimages)
#train_batches = train.shuffle(100).batch(32)
train_batches = data.shuffle(100).batch(32)
validation_batches = val_data.batch(32)
# TRANSFORM PHASE END

In [ ]:
# LOAD PHASE START #
history = model.fit(train_batches, epochs=10, validation_data=validation_batches, validation_steps=1)
# LOAD PHASE END #

### parallelizing

In [33]:
train_data = tfds.load('cats_vs_dogs', split='train', with_info=True)

In [36]:
#file_pattern = f'C:\\Users\\Amanpreet Singh\\tensorflow_datasets\\cats_vs_dogs\\4.0.0\\cats_vs_dogs-train*'
file_pattern = f'C:\\Users\\Amanpreet Singh\\tensorflow_datasets\\cats_vs_dogs\\4.0.0\\cats_vs_dogs-train*'
files = tf.data.Dataset.list_files(file_pattern)

In [37]:
train_dataset = files.interleave(
                        tf.data.TFRecordDataset,
                        cycle_length=4,
                        num_parallel_calls=tf.data.experimental.AUTOTUNE
                        )

In [38]:
def read_tfrecord(serialized_example):
    feature_description={
                            "image": tf.io.FixedLenFeature((), tf.string, ""),
                            "label": tf.io.FixedLenFeature((), tf.int64, -1),
                            }
    example = tf.io.parse_single_example(
                            serialized_example, feature_description
                            )
    image = tf.io.decode_jpeg(example['image'], channels=3)
    image = tf.cast(image, tf.float32)
    image = image / 255
    image = tf.image.resize(image, (300,300))
    return image, example['label']

In [40]:
import multiprocessing
cores = multiprocessing.cpu_count()
print(cores)
train_dataset = train_dataset.map(read_tfrecord, num_parallel_calls=cores)
train_dataset = train_dataset.cache()

4


In [41]:
train_dataset = train_dataset.shuffle(1024).batch(32)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)